In [1]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
import nltk
import string
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from pprint import pprint
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
import pickle
import re, string, unicodedata
import nltk
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from Semi_EM_NB import Semi_EM_MultinomialNB

In [2]:
#Remove metadata 
remove = ('headers', 'footers', 'quotes') 
RANDOM_STATE = 10

# Load dataset
print("Loading 20 newsgroups dataset")
import requests
requests.packages.urllib3.disable_warnings()

import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context
    
newsdata = fetch_20newsgroups(subset='all')
len(newsdata.data)
import nltk
nltk.download('stopwords')
nltk.download('punkt')

Loading 20 newsgroups dataset


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shailajamallick/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/shailajamallick/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
X_remaining, X_test, Y_remaining, Y_test = train_test_split(newsdata.data, newsdata.target, test_size=0.20, random_state=RANDOM_STATE)
sss = StratifiedShuffleSplit(n_splits=1, test_size= 0.625,random_state=RANDOM_STATE)
X_remaining=np.array(X_remaining)
Y_remaining=np.array(Y_remaining)
for train_index, test_index in sss.split(X_remaining,Y_remaining):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_dev = X_remaining[train_index], X_remaining[test_index]
    Y_train, Y_dev = Y_remaining[train_index], Y_remaining[test_index]
    
print('Data loaded')
print()
print('Training data documents:', len(X_train))
print('Development data documents:', len(X_dev))
print('Test data documents:', len(X_test))
print()
print('Total Newsgroups :', newsdata.target_names)

TRAIN: [ 4830  5487  1131 ...  6543 10054  9924] TEST: [ 2647  7876  4857 ... 13387  2034  8962]
Data loaded

Training data documents: 5653
Development data documents: 9423
Test data documents: 3770

Total Newsgroups : ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [4]:
Y_minus=np.full(len(Y_dev), -1, dtype=int)

X_concat=np.concatenate((X_train, X_dev), axis=None)
Y_concat=np.concatenate((Y_train, Y_minus), axis=None)
Y_true=np.concatenate((Y_train, Y_dev), axis=None)
print (X_concat.shape, Y_concat.shape, Y_true.shape)

(15076,) (15076,) (15076,)


In [5]:
def Stem_tokenize(text):
    stemmer = PorterStemmer()
    return [stemmer.stem(w) for w in word_tokenize(text)]

In [6]:
Vect = TfidfVectorizer(analyzer= 'word', tokenizer=Stem_tokenize,
                                stop_words=stopwords.words('english') + list(string.punctuation),
                                lowercase=True, strip_accents='ascii', ngram_range=(1,2),
                                min_df=5, max_df= 0.75)

In [7]:
data_train=Vect.fit_transform(X_concat)
classifier_NB = MultinomialNB(alpha=0.01)
classifier_NB.fit(data_train[:len(X_train)], Y_true[:len(Y_train)])

print ("Multinomial NB. score", classifier_NB.score(data_train[len(X_train):], Y_dev))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", '``', 'abov', 'ani', 'becaus', 'befor', 'could', 'doe', 'dure', 'ha', 'hi', 'might', 'must', "n't", 'need', 'onc', 'onli', 'ourselv', 'sha', 'themselv', 'thi', 'veri', 'wa', 'whi', 'wo', 'would', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Multinomial NB. score 0.8805051469807916


In [8]:
em_nb_clf = Semi_EM_MultinomialNB(alpha=1e-2) # semi supervised EM based Naive Bayes classifier
em_nb_clf.fit(data_train[:len(X_train)], Y_true[:len(Y_train)], data_train[len(X_train):])

pred = em_nb_clf.predict(data_train[len(X_train):])
print("Multinomial NB-EM. score", metrics.accuracy_score(Y_true[len(Y_train):], pred))

Initial expected log likelihood = -15132450.024

EM iteration #1
	Expected log likelihood = -14411634.759
EM iteration #2
	Expected log likelihood = -14411024.838
EM iteration #3
	Expected log likelihood = -14410765.369
EM iteration #4
	Expected log likelihood = -14410765.369
Multinomial NB-EM. score 0.8808235169266688
